In [2]:
from stable_baselines3.common.env_util import make_vec_env
from wandb.integration.sb3 import WandbCallback
from sb3_contrib.ppo_mask import MaskablePPO
from benchmark import get_benchmarking_data
from env import MPSPEnv
import numpy as np
import torch
import wandb
import os
os.environ['WANDB_NOTEBOOK_NAME'] = 'sb3.ipynb'
os.environ["WANDB_SILENT"] = 'true'
wandb.login()

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


True

In [ ]:
from torch import nn

In [ ]:
test = nn.Sequential(
                nn.Flatten(),
                nn.Embedding(
                    7,
                    2
                ),
                nn.Flatten(),
)

In [ ]:
test_tensor = torch.tensor([[[1, 2, 3], [4, 5, 6]], [[4, 2, 3], [4, 5, 1]]])

In [ ]:
square_tensor = torch.tensor([
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]],
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
])
square_tensor

In [ ]:
# Upper triangular matrix (without diagonal)
indeces  = torch.triu_indices(3, 3, offset=1)

In [ ]:
# Extract upper triangular part of square_tensor. Apply along the batch dimension
square_tensor[:, indeces[0], indeces[1]]

In [ ]:
test(test_tensor)

In [4]:
config = {
    # Environment
    'ROWS': 10,
    'COLUMNS': 4,
    'N_PORTS': 10,
    # Model
    'PI_LAYER_SIZES': [64, 128, 64],
    'VF_LAYER_SIZES': [64, 128, 64],
    # Training
    'TOTAL_TIMESTEPS': 4800000,
    'BATCH_SIZE': 128
}

In [ ]:
run = wandb.init(
    project="PPO-SB3",
    entity="rl-msps",
    sync_tensorboard=True,
    name=f"N{config['N_PORTS']}_R{config['ROWS']}_C{config['COLUMNS']}",
    config=config,
    tags=["test"]
)

In [5]:
env = make_vec_env(
    lambda: MPSPEnv(
        config['ROWS'],
        config['COLUMNS'],
        config['N_PORTS']
    ),
    n_envs=8  # M2 with 8 cores
)

In [6]:
env = MPSPEnv(
        config['ROWS'],
        config['COLUMNS'],
        config['N_PORTS']
    )

In [11]:
bay1 = env.observation_space.sample()['bay_matrix']
bay2 = env.observation_space.sample()['bay_matrix']
bay = torch.tensor([bay1, bay2]).float()
container = torch.tensor([[2], [1]]).long()
print(bay)
print(container)


tensor([[[ 9.,  0.,  1.,  4.],
         [ 3.,  9.,  2.,  0.],
         [ 2.,  2.,  0.,  1.],
         [ 5.,  1.,  2.,  0.],
         [ 3.,  8.,  8.,  7.],
         [ 6., 10.,  1.,  3.],
         [ 9.,  9.,  9.,  6.],
         [10.,  8.,  4.,  3.],
         [ 4.,  2.,  9.,  2.],
         [ 1.,  8.,  7.,  0.]],

        [[ 4.,  5.,  8.,  5.],
         [ 9.,  3.,  0.,  8.],
         [ 3.,  9.,  5.,  8.],
         [ 2.,  6.,  5.,  8.],
         [ 7.,  2.,  5.,  2.],
         [ 2.,  9.,  0.,  6.],
         [ 6.,  4.,  6.,  4.],
         [ 8.,  0.,  5.,  1.],
         [ 6.,  7.,  6.,  8.],
         [ 5.,  7., 10.,  6.]]])
tensor([[2],
        [1]])


In [15]:
embed_size = 2
embedding = torch.nn.Embedding(
    config['N_PORTS'],
    embed_size
)

In [77]:
#embedding(container)

tensor([[[ 0.5578,  1.5095]],

        [[-2.1424, -0.7622]]], grad_fn=<EmbeddingBackward0>)

In [68]:
linear_input_size = 10 * embed_size
linear_output_size = 5
linear = torch.nn.Linear(linear_input_size, linear_output_size)

In [8]:
bay

tensor([[[ 4,  4, 10, 10],
         [ 4,  0,  9,  8],
         [ 1,  1,  7,  2],
         [10,  2,  1,  7],
         [ 6,  3,  5,  3],
         [ 8,  1,  7,  2],
         [ 5,  9,  1,  8],
         [ 6, 10,  4,  5],
         [ 4,  5,  0,  8],
         [ 9,  9,  0,  6]],

        [[ 4,  6,  6,  8],
         [10,  5,  3,  7],
         [ 1,  1, 10, 10],
         [ 4,  5,  9,  6],
         [ 2,  2,  2,  5],
         [ 4,  7,  3,  8],
         [ 1,  9,  3,  5],
         [ 7, 10,  0,  2],
         [ 6,  0,  1,  7],
         [ 9,  3,  3,  9]]])

In [9]:
linear = torch.nn.Linear(4, 2)

In [36]:
a = torch.arange(10).repeat(2,1)
a

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [31]:
embedding(a)

tensor([[[ 0.0294,  0.9602],
         [-0.5457,  1.2565],
         [-0.7554,  0.4130],
         [-0.1260, -0.1035],
         [ 1.3723, -0.1461],
         [ 1.6528,  1.4048],
         [-0.7836, -0.2007],
         [ 0.9972,  0.7696],
         [ 0.3399, -1.9451],
         [-0.4238, -0.4082]],

        [[ 0.0294,  0.9602],
         [-0.5457,  1.2565],
         [-0.7554,  0.4130],
         [-0.1260, -0.1035],
         [ 1.3723, -0.1461],
         [ 1.6528,  1.4048],
         [-0.7836, -0.2007],
         [ 0.9972,  0.7696],
         [ 0.3399, -1.9451],
         [-0.4238, -0.4082]]], grad_fn=<EmbeddingBackward0>)

In [19]:
# As long
embedding(torch.arange(10).long())

tensor([[ 0.0294,  0.9602],
        [-0.5457,  1.2565],
        [-0.7554,  0.4130],
        [-0.1260, -0.1035],
        [ 1.3723, -0.1461],
        [ 1.6528,  1.4048],
        [-0.7836, -0.2007],
        [ 0.9972,  0.7696],
        [ 0.3399, -1.9451],
        [-0.4238, -0.4082]], grad_fn=<EmbeddingBackward0>)

In [38]:
torch.cat([linear(bay), embedding(a)], dim=2)

tensor([[[ 0.9858,  1.3615,  0.0294,  0.9602],
         [ 4.8792, -4.2454, -0.5457,  1.2565],
         [ 1.4597, -0.7545, -0.7554,  0.4130],
         [ 0.9516, -0.3662, -0.1260, -0.1035],
         [ 9.0474, -5.8051,  1.3723, -0.1461],
         [ 5.6129, -3.6134,  1.6528,  1.4048],
         [ 8.6722, -5.2568, -0.7836, -0.2007],
         [ 5.2210, -2.9066,  0.9972,  0.7696],
         [ 4.7701, -3.4271,  0.3399, -1.9451],
         [ 6.5015, -6.0037, -0.4238, -0.4082]],

        [[ 6.8351, -4.3232,  0.0294,  0.9602],
         [ 3.4030,  0.4797, -0.5457,  1.2565],
         [ 8.7764, -5.1753, -0.7554,  0.4130],
         [ 7.5482, -4.1114, -0.1260, -0.1035],
         [ 2.9118, -1.3645,  1.3723, -0.1461],
         [ 6.4130, -3.6865,  1.6528,  1.4048],
         [ 5.0278, -2.7655, -0.7836, -0.2007],
         [ 1.5078, -0.2974,  0.9972,  0.7696],
         [ 7.8048, -3.9955,  0.3399, -1.9451],
         [ 8.6791, -5.6400, -0.4238, -0.4082]]], grad_fn=<CatBackward0>)

In [37]:
embedding(a)

tensor([[[ 0.0294,  0.9602],
         [-0.5457,  1.2565],
         [-0.7554,  0.4130],
         [-0.1260, -0.1035],
         [ 1.3723, -0.1461],
         [ 1.6528,  1.4048],
         [-0.7836, -0.2007],
         [ 0.9972,  0.7696],
         [ 0.3399, -1.9451],
         [-0.4238, -0.4082]],

        [[ 0.0294,  0.9602],
         [-0.5457,  1.2565],
         [-0.7554,  0.4130],
         [-0.1260, -0.1035],
         [ 1.3723, -0.1461],
         [ 1.6528,  1.4048],
         [-0.7836, -0.2007],
         [ 0.9972,  0.7696],
         [ 0.3399, -1.9451],
         [-0.4238, -0.4082]]], grad_fn=<EmbeddingBackward0>)

In [12]:
linear(bay)

tensor([[[ 0.9858,  1.3615],
         [ 4.8792, -4.2454],
         [ 1.4597, -0.7545],
         [ 0.9516, -0.3662],
         [ 9.0474, -5.8051],
         [ 5.6129, -3.6134],
         [ 8.6722, -5.2568],
         [ 5.2210, -2.9066],
         [ 4.7701, -3.4271],
         [ 6.5015, -6.0037]],

        [[ 6.8351, -4.3232],
         [ 3.4030,  0.4797],
         [ 8.7764, -5.1753],
         [ 7.5482, -4.1114],
         [ 2.9118, -1.3645],
         [ 6.4130, -3.6865],
         [ 5.0278, -2.7655],
         [ 1.5078, -0.2974],
         [ 7.8048, -3.9955],
         [ 8.6791, -5.6400]]], grad_fn=<ViewBackward0>)

In [76]:
linear(output)

tensor([[[-0.6926, -0.5299,  0.0261,  1.6368,  0.0278],
         [ 0.0350, -0.2245, -0.3679,  0.2331, -0.4008],
         [ 0.3575,  0.7104,  0.1020,  0.2330, -0.4625],
         [ 0.7231, -0.1233, -0.3874,  0.3690, -0.5451]],

        [[ 0.5493,  1.0466,  0.5138, -0.3125,  1.0142],
         [-0.4114,  0.1759, -0.1001, -0.2051, -1.0957],
         [-0.7512, -0.5073,  0.8956,  0.6195, -0.0052],
         [ 0.7329, -0.3540, -0.1081, -0.1655,  0.1595]]],
       grad_fn=<ViewBackward0>)

In [74]:
torch.nn.Flatten()(linear(output))

tensor([[-0.6926, -0.5299,  0.0261,  1.6368,  0.0278,  0.0350, -0.2245, -0.3679,
          0.2331, -0.4008,  0.3575,  0.7104,  0.1020,  0.2330, -0.4625,  0.7231,
         -0.1233, -0.3874,  0.3690, -0.5451],
        [ 0.5493,  1.0466,  0.5138, -0.3125,  1.0142, -0.4114,  0.1759, -0.1001,
         -0.2051, -1.0957, -0.7512, -0.5073,  0.8956,  0.6195, -0.0052,  0.7329,
         -0.3540, -0.1081, -0.1655,  0.1595]], grad_fn=<ReshapeAliasBackward0>)

In [ ]:
env._get_long_distance_transportation_matrix(10)

In [ ]:
env._get_short_distance_transportation_matrix(10)

In [ ]:
env._get_mixed_distance_transportation_matrix(10)

In [ ]:
env = MPSPEnv(
        config['ROWS'],
        config['COLUMNS'],
        config['N_PORTS']
)

In [ ]:
observation = env.reset()

In [ ]:
import torch.nn as nn

In [ ]:
conv1 = nn.Conv2d(
    in_channels=1,
    out_channels=3,
    kernel_size=3,
    stride=1,
    padding=1
)

In [ ]:
input = torch.tensor(observation['bay_matrix'], dtype=torch.float32).unsqueeze(0)
input

In [ ]:
input = conv1(input)
input

In [ ]:
pool = nn.MaxPool2d(2, 2)
input = pool(input)
input

In [ ]:
observation

In [ ]:
policy_kwargs = {
    'activation_fn': torch.nn.ReLU,
    'net_arch': [{
        'pi': config['PI_LAYER_SIZES'],
        'vf': config['VF_LAYER_SIZES']
    }]
}

wandb_run_path = None

if wandb_run_path:
    model_file = wandb.restore('model.zip', run_path=wandb_run_path)
    model = MaskablePPO.load(
        model_file.name,
        env=env
    )
else:
    model = MaskablePPO(
        policy='MultiInputPolicy',
        env=env,
        batch_size=config['BATCH_SIZE'],
        verbose=0,
        # tensorboard_log=f"runs/{run.id}",
        policy_kwargs=policy_kwargs
    )

In [ ]:
vars(model)

In [ ]:
model.learn(
    total_timesteps=config['TOTAL_TIMESTEPS'],
    callback=WandbCallback(
        model_save_path=f"models/{run.id}",
        model_save_freq=config['TOTAL_TIMESTEPS'] // 4,
    )
)

In [ ]:
eval_data = get_benchmarking_data('rl-mpsp-benchmark/set_2')
eval_data = [
    e for e in eval_data if (
        e['R'] == config['ROWS'] and
        e['C'] == config['COLUMNS'] and
        e['N'] == config['N_PORTS']
    )
]

In [ ]:
# Creating seperate env for evaluation
env = MPSPEnv(
    config['ROWS'],
    config['COLUMNS'],
    config['N_PORTS']
)

eval_rewards = []
# Negative because env returns negative reward for shifts
paper_rewards = [-e['paper_result'] for e in eval_data]
paper_seeds = [e['seed'] for e in eval_data]

for e in eval_data:
    total_reward = 0
    obs = env.reset(
        transportation_matrix=e['transportation_matrix']
    )
    done = False
    while not done:
        action, _ = model.predict(
            obs,
            action_masks=env.action_masks()
        )
        obs, reward, done, _ = env.step(action)
        total_reward += reward

    eval_rewards.append(total_reward)

eval = {
    'mean_reward': np.mean(eval_rewards),
    'mean_paper_reward': np.mean(paper_rewards),
    'rewards': eval_rewards,
    'paper_rewards': paper_rewards,
    'paper_seeds': paper_seeds
}
run.summary['evaluation_benchmark'] = eval

In [ ]:
run.finish()